# Integrentes:

* `Andresa Bicudo`
* `Gabriel Yamashita`
* `Leonardo Malta`

# Introdução 
A pandemia do corona vírus fez com que o futebol parasse, inclusive o campeonato inglês (Premiere League). Existem campeonatos que foram interrompidos, como o campeonato francês, mesmo sem ter finalizado todos os jogos e também há campeonatos que retonaram porém com desempenhos diferentes de cada equipe. Isso fez com que surgisse o seguinte questionamento: o que provavelmente ocorreria com o campeonato inglês se a pandemia não tivesse interrompido o campeonato?

# Objetivo:
Prever o restante do campeonato inglês com base no desempenho e estatísitcas das equipes até a 30ª rodada. 

# Separação dos Dados:

In [65]:
# Importanto Bibliotecas para o Trabalho:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import statsmodels.api as sm

import scipy as scp
import sklearn

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn import metrics
from sklearn.metrics import confusion_matrix

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [66]:
dados = pd.read_excel('masterdata.xlsx')
dados.head(5)

,MatchID,Home_team,Away_team,Score_home,Score_away,Possession_home,Possession_away,Shots_on_target_home,Shots_on_target_away,Shots_home,Shots_away,Touches_home,Touches_away,Passes_home,Passes_away,Tackles_home,Tackles_away,Clearances_home,Clearances_away,Corners_home,Corners_away,Offsides_home,Offsides_away,Yellow_cards_home,Yellow_cards_away,Fouls_conceded_home,Fouls_conceded_away,Red_cards_home,Red_cards_away,year,Expenditures_home,Arrivals_home,Income_home,Departures_home,Expenditures_away,Arrivals_away,Income_away,Departures_away
0,5937,Blackburn,Reading,3,3,54.0,46.0,6,4,15,11,553,464,356,287,22,13,33,19,5,10,5,3,2,0,18,7,0,0,2007,18.84,21,19.28,25,10.52,25,0.00,19
1,5938,Bolton,Aston Villa,2,2,47.1,52.9,2,2,11,7,457,493,318,343,24,15,23,20,7,6,0,2,2,1,10,11,0,0,2007,19.38,16,4.39,16,28.16,17,2.08,18
2,5939,Chelsea,Everton,1,1,59.3,40.7,7,6,20,11,615,471,425,305,23,22,10,28,10,2,6,2,2,1,13,7,0,0,2007,101.43,23,59.96,22,20.41,11,2.85,16
3,5940,Liverpool,Charlton,2,2,61.6,38.4,5,4,23,9,632,420,453,271,20,14,22,29,9,2,6,4,0,0,5,13,0,0,2007,52.44,27,23.94,28,24.68,31,2.59,36
4,5941,Man Utd,West Ham,0,1,65.3,34.7,7,2,30,8,825,506,619,333,28,26,18,47,14,3,0,1,0,2,13,12,0,0,2007,31.01,25,20.52,29,79.63,22,7.70,23


In [69]:
def Determina_Resultado(df, home, away):
    if df[home] > df[away]:
        return 'H'
    elif df[home] < df[away]:
        return 'A'
    elif df[home] == df[away]:
        return 'D'


dados['Result'] = dados.apply(
    lambda dados: Determina_Resultado(dados, 'Score_home', 'Score_away'), axis=1)
dados.head(5)

,MatchID,Home_team,Away_team,Score_home,Score_away,Possession_home,Possession_away,Shots_on_target_home,Shots_on_target_away,Shots_home,Shots_away,Touches_home,Touches_away,Passes_home,Passes_away,Tackles_home,Tackles_away,Clearances_home,Clearances_away,Corners_home,Corners_away,Offsides_home,Offsides_away,Yellow_cards_home,Yellow_cards_away,Fouls_conceded_home,Fouls_conceded_away,Red_cards_home,Red_cards_away,year,Expenditures_home,Arrivals_home,Income_home,Departures_home,Expenditures_away,Arrivals_away,Income_away,Departures_away,Result
0,5937,Blackburn,Reading,3,3,54.0,46.0,6,4,15,11,553,464,356,287,22,13,33,19,5,10,5,3,2,0,18,7,0,0,2007,18.84,21,19.28,25,10.52,25,0.00,19,D
1,5938,Bolton,Aston Villa,2,2,47.1,52.9,2,2,11,7,457,493,318,343,24,15,23,20,7,6,0,2,2,1,10,11,0,0,2007,19.38,16,4.39,16,28.16,17,2.08,18,D
2,5939,Chelsea,Everton,1,1,59.3,40.7,7,6,20,11,615,471,425,305,23,22,10,28,10,2,6,2,2,1,13,7,0,0,2007,101.43,23,59.96,22,20.41,11,2.85,16,D
3,5940,Liverpool,Charlton,2,2,61.6,38.4,5,4,23,9,632,420,453,271,20,14,22,29,9,2,6,4,0,0,5,13,0,0,2007,52.44,27,23.94,28,24.68,31,2.59,36,D
4,5941,Man Utd,West Ham,0,1,65.3,34.7,7,2,30,8,825,506,619,333,28,26,18,47,14,3,0,1,0,2,13,12,0,0,2007,31.01,25,20.52,29,79.63,22,7.70,23,A


# Obtendo Variáveis de Interesse  

In [72]:
# Definindo variáveis imortantes: 
stats = dados.drop(columns=[
    'MatchID', 'Home_team', 'Away_team', 'Score_home', 'Score_away',
    'year', 'Result'
])

stats.head(5)

,Possession_home,Possession_away,Shots_on_target_home,Shots_on_target_away,Shots_home,Shots_away,Touches_home,Touches_away,Passes_home,Passes_away,Tackles_home,Tackles_away,Clearances_home,Clearances_away,Corners_home,Corners_away,Offsides_home,Offsides_away,Yellow_cards_home,Yellow_cards_away,Fouls_conceded_home,Fouls_conceded_away,Red_cards_home,Red_cards_away,Expenditures_home,Arrivals_home,Income_home,Departures_home,Expenditures_away,Arrivals_away,Income_away,Departures_away
0,54.0,46.0,6,4,15,11,553,464,356,287,22,13,33,19,5,10,5,3,2,0,18,7,0,0,18.84,21,19.28,25,10.52,25,0.00,19
1,47.1,52.9,2,2,11,7,457,493,318,343,24,15,23,20,7,6,0,2,2,1,10,11,0,0,19.38,16,4.39,16,28.16,17,2.08,18
2,59.3,40.7,7,6,20,11,615,471,425,305,23,22,10,28,10,2,6,2,2,1,13,7,0,0,101.43,23,59.96,22,20.41,11,2.85,16
3,61.6,38.4,5,4,23,9,632,420,453,271,20,14,22,29,9,2,6,4,0,0,5,13,0,0,52.44,27,23.94,28,24.68,31,2.59,36
4,65.3,34.7,7,2,30,8,825,506,619,333,28,26,18,47,14,3,0,1,0,2,13,12,0,0,31.01,25,20.52,29,79.63,22,7.70,23


In [ ]:
#data.loc[dados.FTR == 'A', 'FTR'] = 0
#data.loc[dados.FTR == 'D', 'FTR'] = 1
#data.loc[dados.FTR == 'H', 'FTR'] = 2

games = data.loc[: , ['HomeTeam', 'AwayTeam']]
stats = data.drop(['HomeTeam', 'AwayTeam', 'HTHG', 'HTAG', 'HTR'], axis=1)
stats.head(5)

A partir da análise desses gráficos de pizza e dos gráficos de destribuição, nota-se que a chance de um time ganhar e perder no seu estádio, no geral, são similares. Isso leva a conclusão de que isso varia de time para time. Por exemplo, pode haver um time que jogando fora do seu estádio contra determina equipe e conseguindo adquirir resultados bons, mas isso não acontece no geral, o que nos leva a concluir que cada confronto possuem suas peculiaridades e variáveis de interesse. 

In [ ]:
def Modelo(home, away, df):
    index = games.loc[(games['HomeTeam'] == home) &
                      (games['AwayTeam'] == away), :].index
    x_treinamento = df.iloc[index].drop(['FTR'], axis=1)
    y_treinamento = df.FTR.iloc[index]
    
    #logit_model = sm.MNLogit(y_treinamento, sm.add_constant(x_treinamento))
    #result = logit_model.fit()
    #stats1 = result.summary()
    #stats2 = result.summary2()
    
    return [x_treinamento, y_treinamento]


print(Modelo('Man City', 'Man United', stats)[0])
print('----------------------------------------------------------------------------')
print(Modelo('Man City', 'Man United', stats)[1])

In [ ]:
# Removendo dados por meio do valor p:
logit_model = sm.MNLogit(
    Modelo('Liverpool', 'Man City', stats)[1],
    sm.add_constant(Modelo('Liverpool', 'Man City', stats)[0]))

result = logit_model.fit()
stats1 = result.summary()
stats2 = result.summary2()
print(stats1)

In [ ]:
print(stats2)

In [ ]:
# Resultados
result.predict(sm.add_constant(x_treinamento))